# Sparse Optical Flow-based Data Generation
A fully automatic data generation, relying on the generated ROI_data, without the need of creating the ROIs runtime.

In [1]:
saveRawOF = True #True for saving .mat files from the raw OF values, False for the histogram creation as before

saveCSV = True #Save as CSV file instead of .mat

separateGestures = False #True for Gesture Separation, False for processing each video as a whole

sample_size = 30 #MAKE SURE IT'S THE SAME AS 'cornerNum' WAS WHEN THE ROI_data WAS GENERATED

modes = [ "Sparse_OF", "Sparse_Points"]

mode = modes[0]

outfolder_root = '../data/' #The folder path for storing the .mat files
root = '../data/JIGSAWS/' #The folder path for the JIGSAWS dataset's root


#The actual path and filename is decided by these,
#and in the next step created automatically if they don't yet exist

if separateGestures and saveRawOF:
    output_root = outfolder_root + 'RAW_Separate_Gestures_' + mode + '/'
elif separateGestures:
    output_root = outfolder_root + 'Separate_Features_' + mode + '/'
elif saveRawOF and saveCSV:
    output_root = root
elif saveRawOF:
    output_root = outfolder_root + 'RAW_' + mode + '_full_by_frames/'
else:
    output_root = outfolder_root + 'Global_Separate_Features_' + mode + '/'

outfile_base = '2Features_' + mode + '_' # + Skill + .mat added at the function call

In [2]:
import os
# Create directory
dirName = output_root
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  /home/gabor/Asztal/IROB/Thesis_resources/JIGSAW/  already exists


In [3]:
import cv2 as cv
import numpy as np

skills = ['Knot_Tying', 'Needle_Passing', 'Suturing']


# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

feature_params = dict( maxCorners = sample_size,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )


color = np.random.randint(0,255,(30,3))


def getJIGSAWvideoTitles():
    dir_name = root + skills[0] + '/video/'
    knot_list = [os.path.basename(x) for x in glob.glob(dir_name+'*.avi')]
    dir_name = root + skills[1] + '/video/'
    needle_list = [os.path.basename(x) for x in glob.glob(dir_name+'*.avi')]
    dir_name = root + skills[2] + '/video/'
    suture_list = [os.path.basename(x) for x in glob.glob(dir_name+'*.avi')]
    knot_list.sort()
    needle_list.sort()
    suture_list.sort()
    return knot_list, needle_list, suture_list

In [4]:
import os
import glob

knot_list, needle_list, suture_list = getJIGSAWvideoTitles()
print(knot_list[0])
print(needle_list[0])
print(suture_list[0])

Knot_Tying_B001_capture1.avi
Needle_Passing_B001_capture1.avi
Suturing_B001_capture1.avi


In [5]:
def getJIGSAWgestureList():
    dir_name = root + skills[0] + '/transcriptions/'
    trans_list = [os.path.basename(x) for x in glob.glob(dir_name+'*.txt')]
    dir_name = root + skills[1] + '/transcriptions/'
    trans_list.extend([os.path.basename(x) for x in glob.glob(dir_name+'*.txt')])
    dir_name = root + skills[2] + '/transcriptions/'
    trans_list.extend([os.path.basename(x) for x in glob.glob(dir_name+'*.txt')])
    trans_list.sort()
    print(len(trans_list))
    return trans_list

In [6]:
transc_list = getJIGSAWgestureList()
print(transc_list[0])

103
Knot_Tying_B001.txt


In [7]:
def getROIdata(skill, user, attempt, capture):
    base = root + skill + '/ROI_data/' + skill + '_' + user + attempt + '_' + capture
    path = base + '.npy'
    data = np.load(path)
    path = base + '_frame_index.txt'
    f = open(path, "r")
    frame_ind = f.read()
    f.close()
    return data, frame_ind

In [8]:
def parseFileName(filename):
    var = filename
    index = 0
    skill = var[index] #gathering skill-string
    index += 1
    while(var[index] != '_'):
        skill += var[index]
        index += 1 #finding first underscore
    if filename[0] != 'S': #in case of Suturing there is one less underscore
        skill += var[index]
        index += 1
        while(var[index] != '_'):
            skill += var[index]
            index += 1
    index += 1 #skipping second underscore
    user = var[index] #saving user_id
    index += 1 #skipping userID
    attempt = var[index] #first digit of attempt id
    index += 1
    while(var[index].isdigit()):
        attempt += var[index]
        index += 1
    while(var[index] != '.'):
        index += 1
    capture = var[index-1] #capture_num is right before the dot
    return  user, attempt, capture, skill

In [9]:
def getGestureSeparatedFrameRanges(filename):
    gestures = []
    frame_begin = []
    frame_end = []
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            i = 0
            temp = ""
            while line[i] != ' ':
                temp += line[i]
                i += 1
            frame_begin.append(int(temp)) #first characters form the beginning frames
            temp = ""
            while line[i] == ' ':
                i += 1 #skip spaces
            while line[i] != ' ':
                temp += line[i]
                i += 1
            frame_end.append(int(temp)) #second is the end frame
            temp = ""
            while line[i] == ' ':
                i += 1 #skip spaces
            while line[i] != ' ' and line[i] != '\n':
                temp += line[i]
                i += 1
            gestures.append(temp)
    output = np.vstack((np.array(gestures), np.array(frame_begin), np.array(frame_end)))
    f.close()
    return output

In [10]:
def TranscriptionPath(user, attempt, skill):
    path = root + skill + '/transcriptions/'+ skill + '_' + user +  attempt + '.txt'
    return path

In [11]:
def get_optical_flow_at_given_frame_with_ROI_data(full_path, frame_begin, frame_end, p0,frame_range):
    mean_dxs = []
    mean_dys = []
    raw_out = []
    flow = p0
    cap = cv.VideoCapture(full_path)
    cap.set(cv.CAP_PROP_POS_FRAMES, int(frame_begin)-1)
    ret, frame1 = cap.read()
    if not ret:
        print('input error at: ' + video_filename)
        return
    prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
    curr_frame_num = cap.get(cv.CAP_PROP_POS_FRAMES)
    ind = 0
    while(int(curr_frame_num) <= int(frame_end)):
        ret, frame2 = cap.read()
        curr_frame_num = cap.get(cv.CAP_PROP_POS_FRAMES);
        if not ret or (frame_range != 0 and ind == frame_range):
            break

        next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
        p0_old = p0
        p0_1 = p0[:,0,:].reshape(-1,1,2).astype(np.float32)
        feat1_new, _, _ = cv.calcOpticalFlowPyrLK(prvs, next, p0_1, None, **lk_params)
        p0_2 = p0[:,0,:].reshape(-1,1,2).astype(np.float32)
        feat2_new,_,_ = cv.calcOpticalFlowPyrLK(prvs, next, p0_2, None, **lk_params)
        prvs = next.copy()
        p0 = np.concatenate((feat1_new, feat2_new), axis=1).astype(np.float32)
        if mode == "Sparse_OF":
            feat1_old = p0_old[:,0,:]
            feat1_old = np.expand_dims(feat1_old, axis=1)
            feat2_old = p0_old[:,1,:]
            feat2_old = np.expand_dims(feat2_old, axis=1)
            diff_1 = np.subtract(feat1_old, feat1_new)
            diff_2 = np.subtract(feat2_old, feat2_new)
            diff = np.asarray(np.concatenate((diff_1, diff_2), axis=1)).astype(np.float32)
            flow = np.asarray(np.concatenate((diff, p0), axis = 2)).astype(np.float32)
        elif mode == "Sparse_Points":
            flow = p0

        if saveRawOF:
            raw_out.append(flow)
        else:
            dx, dy = np.reshape(np.swapaxes(flow,0,2), (2, -1))

            mean_dxs.append(np.mean(dx))
            mean_dys.append(np.mean(dy))
        ind += 1

    cap.release()
    if saveRawOF:
        return raw_out
    else:
        return mean_dxs, mean_dys, p0

In [12]:
def extractDataFromSpecificFrames(full_path, filename, gesture, frame_begin, frame_end, p0, frame_range):
    if saveRawOF:
        flow = get_optical_flow_at_given_frame_with_ROI_data(
                                        full_path,frame_begin, frame_end,p0,frame_range)
        return flow
    else:
        mean_dxs, mean_dys, p0 = get_optical_flow_at_given_frame_with_ROI_data(
                                        full_path,frame_begin, frame_end,p0,frame_range)
        x = np.array([mean_dxs])
        y = np.array([mean_dys])
        histo = np.hstack((np.array(x), np.array(y))).astype(np.object)
        #print('Histogram extracted from ' + filename +
        #      ' for gesture (' + gesture  + ') from frames: ' +str(frame_begin) + '-' + str(frame_end))
        return histo, p0

In [ ]:
def saveCSVs(skill, user, attempt, capture, data):
    skill_short = "short"
    if skill == 'Knot_Tying':
        skill_short = 'Knot'
    if skill == 'Needle_Passing':
        skill_short = 'Needle'
    if skill == 'Suturing':
        skill_short = 'Suture'
    filename = output_root + skill + '/video/csvs/' + skill_short + '_'
    filename = filename +  user + '_' + attempt + '_' + capture + '.csv'
    #data.shape = frame_numberx30x2x2
    with open(filename, 'w', newline='') as csvfile:
        #defining the field names,
        #namely the 30 points of interest for the two features, and their two data members
        #This defines the possible coloumns
        fieldnames = [#the OF value pairs for each feature
                      'feature_1_1_1', 'feature_1_1_2','feature_2_1_1', 'feature_2_1_2',
                      'feature_1_2_1', 'feature_1_2_2', 'feature_2_2_1', 'feature_2_2_2',
                      'feature_1_3_1', 'feature_1_3_2', 'feature_2_3_1', 'feature_2_3_2',
                      'feature_1_4_1', 'feature_1_4_2', 'feature_2_4_1', 'feature_2_4_2',
                      'feature_1_5_1', 'feature_1_5_2', 'feature_2_5_1', 'feature_2_5_2',
                      'feature_1_6_1', 'feature_1_6_2', 'feature_2_6_1', 'feature_2_6_2',
                      'feature_1_7_1', 'feature_1_7_2', 'feature_2_7_1', 'feature_2_7_2',
                      'feature_1_8_1', 'feature_1_8_2', 'feature_2_8_1', 'feature_2_8_2',
                      'feature_1_9_1', 'feature_1_9_2', 'feature_2_9_1', 'feature_2_9_2',
                      'feature_1_10_1', 'feature_1_10_2', 'feature_2_10_1', 'feature_2_10_2',
                      'feature_1_11_1', 'feature_1_11_2', 'feature_2_11_1', 'feature_2_11_2',
                      'feature_1_12_1', 'feature_1_12_2', 'feature_2_12_1', 'feature_2_12_2',
                      'feature_1_13_1', 'feature_1_13_2', 'feature_2_13_1', 'feature_2_13_2',
                      'feature_1_14_1', 'feature_1_14_2', 'feature_2_14_1', 'feature_2_14_2',
                      'feature_1_15_1', 'feature_1_15_2', 'feature_2_15_1', 'feature_2_15_2',
                      'feature_1_16_1', 'feature_1_16_2', 'feature_2_16_1', 'feature_2_16_2',
                      'feature_1_17_1', 'feature_1_17_2', 'feature_2_17_1', 'feature_2_17_2',
                      'feature_1_18_1', 'feature_1_18_2', 'feature_2_18_1', 'feature_2_18_2',
                      'feature_1_19_1', 'feature_1_19_2', 'feature_2_19_1', 'feature_2_19_2',
                      'feature_1_20_1', 'feature_1_20_2', 'feature_2_20_1', 'feature_2_20_2',
                      'feature_1_21_1', 'feature_1_21_2', 'feature_2_21_1', 'feature_2_21_2',
                      'feature_1_22_1', 'feature_1_22_2', 'feature_2_22_1', 'feature_2_22_2',
                      'feature_1_23_1', 'feature_1_23_2', 'feature_2_23_1', 'feature_2_23_2',
                      'feature_1_24_1', 'feature_1_24_2', 'feature_2_24_1', 'feature_2_24_2',
                      'feature_1_25_1', 'feature_1_25_2', 'feature_2_25_1', 'feature_2_25_2',
                      'feature_1_26_1', 'feature_1_26_2', 'feature_2_26_1', 'feature_2_26_2',
                      'feature_1_27_1', 'feature_1_27_2', 'feature_2_27_1', 'feature_2_27_2',
                      'feature_1_28_1', 'feature_1_28_2', 'feature_2_28_1', 'feature_2_28_2',
                      'feature_1_29_1', 'feature_1_29_2', 'feature_2_29_1', 'feature_2_29_2',
                      'feature_1_30_1', 'feature_1_30_2', 'feature_2_30_1', 'feature_2_30_2',
                       #The tracked position values (x,y coordinates) of each feature
                      'pos_1_1_1', 'pos_1_1_2','pos_2_1_1', 'pos_2_1_2',
                      'pos_1_2_1', 'pos_1_2_2', 'pos_2_2_1', 'pos_2_2_2',
                      'pos_1_3_1', 'pos_1_3_2', 'pos_2_3_1', 'pos_2_3_2',
                      'pos_1_4_1', 'pos_1_4_2', 'pos_2_4_1', 'pos_2_4_2',
                      'pos_1_5_1', 'pos_1_5_2', 'pos_2_5_1', 'pos_2_5_2',
                      'pos_1_6_1', 'pos_1_6_2', 'pos_2_6_1', 'pos_2_6_2',
                      'pos_1_7_1', 'pos_1_7_2', 'pos_2_7_1', 'pos_2_7_2',
                      'pos_1_8_1', 'pos_1_8_2', 'pos_2_8_1', 'pos_2_8_2',
                      'pos_1_9_1', 'pos_1_9_2', 'pos_2_9_1', 'pos_2_9_2',
                      'pos_1_10_1', 'pos_1_10_2', 'pos_2_10_1', 'pos_2_10_2',
                      'pos_1_11_1', 'pos_1_11_2', 'pos_2_11_1', 'pos_2_11_2',
                      'pos_1_12_1', 'pos_1_12_2', 'pos_2_12_1', 'pos_2_12_2',
                      'pos_1_13_1', 'pos_1_13_2', 'pos_2_13_1', 'pos_2_13_2',
                      'pos_1_14_1', 'pos_1_14_2', 'pos_2_14_1', 'pos_2_14_2',
                      'pos_1_15_1', 'pos_1_15_2', 'pos_2_15_1', 'pos_2_15_2',
                      'pos_1_16_1', 'pos_1_16_2', 'pos_2_16_1', 'pos_2_16_2',
                      'pos_1_17_1', 'pos_1_17_2', 'pos_2_17_1', 'pos_2_17_2',
                      'pos_1_18_1', 'pos_1_18_2', 'pos_2_18_1', 'pos_2_18_2',
                      'pos_1_19_1', 'pos_1_19_2', 'pos_2_19_1', 'pos_2_19_2',
                      'pos_1_20_1', 'pos_1_20_2', 'pos_2_20_1', 'pos_2_20_2',
                      'pos_1_21_1', 'pos_1_21_2', 'pos_2_21_1', 'pos_2_21_2',
                      'pos_1_22_1', 'pos_1_22_2', 'pos_2_22_1', 'pos_2_22_2',
                      'pos_1_23_1', 'pos_1_23_2', 'pos_2_23_1', 'pos_2_23_2',
                      'pos_1_24_1', 'pos_1_24_2', 'pos_2_24_1', 'pos_2_24_2',
                      'pos_1_25_1', 'pos_1_25_2', 'pos_2_25_1', 'pos_2_25_2',
                      'pos_1_26_1', 'pos_1_26_2', 'pos_2_26_1', 'pos_2_26_2',
                      'pos_1_27_1', 'pos_1_27_2', 'pos_2_27_1', 'pos_2_27_2',
                      'pos_1_28_1', 'pos_1_28_2', 'pos_2_28_1', 'pos_2_28_2',
                      'pos_1_29_1', 'pos_1_29_2', 'pos_2_29_1', 'pos_2_29_2',
                      'pos_1_30_1', 'pos_1_30_2', 'pos_2_30_1', 'pos_2_30_2'
                     ]
        writer = csv.DictWriter(csvfile,fieldnames=fieldnames)
        #writer.writeheader() #uncomment if you want to include the coloumn titles in the first row
        for d in data: #30x2x4 (now also containing the coordinates/position info)
            #create a temporary dictionary for the given frame's data
            #This basically creates the datarow, by order of the coloumns
            temp = {
            #OF values of the first feature (tool_1)
            'feature_1_1_1': d[0][0][0], 'feature_1_1_2': d[0][0][1],
            'feature_1_2_1': d[1][0][0], 'feature_1_2_2': d[1][0][1],
            'feature_1_3_1': d[2][0][0], 'feature_1_3_2': d[2][0][1],
            'feature_1_4_1': d[3][0][0], 'feature_1_4_2': d[3][0][1],
            'feature_1_5_1': d[4][0][0], 'feature_1_5_2': d[4][0][1],
            'feature_1_6_1': d[5][0][0], 'feature_1_6_2': d[5][0][1],
            'feature_1_7_1': d[6][0][0], 'feature_1_7_2': d[6][0][1],
            'feature_1_8_1': d[7][0][0], 'feature_1_8_2': d[7][0][1],
            'feature_1_9_1': d[8][0][0], 'feature_1_9_2': d[8][0][1],
            'feature_1_10_1': d[9][0][0], 'feature_1_10_2': d[9][0][1],
            'feature_1_11_1': d[10][0][0], 'feature_1_11_2': d[10][0][1],
            'feature_1_12_1': d[11][0][0], 'feature_1_12_2': d[11][0][1],
            'feature_1_13_1': d[12][0][0], 'feature_1_13_2': d[12][0][1],
            'feature_1_14_1': d[13][0][0], 'feature_1_14_2': d[13][0][1],
            'feature_1_15_1': d[14][0][0], 'feature_1_15_2': d[14][0][1],
            'feature_1_16_1': d[15][0][0], 'feature_1_16_2': d[15][0][1],
            'feature_1_17_1': d[16][0][0], 'feature_1_17_2': d[16][0][1],
            'feature_1_18_1': d[17][0][0], 'feature_1_18_2': d[17][0][1],
            'feature_1_19_1': d[18][0][0], 'feature_1_19_2': d[18][0][1],
            'feature_1_20_1': d[19][0][0], 'feature_1_20_2': d[19][0][1],
            'feature_1_21_1': d[20][0][0], 'feature_1_21_2': d[20][0][1],
            'feature_1_22_1': d[21][0][0], 'feature_1_22_2': d[21][0][1],
            'feature_1_23_1': d[22][0][0], 'feature_1_23_2': d[22][0][1],
            'feature_1_24_1': d[23][0][0], 'feature_1_24_2': d[23][0][1],
            'feature_1_25_1': d[24][0][0], 'feature_1_25_2': d[24][0][1],
            'feature_1_26_1': d[25][0][0], 'feature_1_26_2': d[25][0][1],
            'feature_1_27_1': d[26][0][0], 'feature_1_27_2': d[26][0][1],
            'feature_1_28_1': d[27][0][0], 'feature_1_28_2': d[27][0][1],
            'feature_1_29_1': d[28][0][0], 'feature_1_29_2': d[28][0][1],
            'feature_1_30_1': d[29][0][0], 'feature_1_30_2': d[29][0][1],
            #Coordinates of the first feature (tool_1)
            'pos_1_1_1': d[0][0][2], 'pos_1_1_2': d[0][0][3],
            'pos_1_2_1': d[1][0][2], 'pos_1_2_2': d[1][0][3],
            'pos_1_3_1': d[2][0][2], 'pos_1_3_2': d[2][0][3],
            'pos_1_4_1': d[3][0][2], 'pos_1_4_2': d[3][0][3],
            'pos_1_5_1': d[4][0][2], 'pos_1_5_2': d[4][0][3],
            'pos_1_6_1': d[5][0][2], 'pos_1_6_2': d[5][0][3],
            'pos_1_7_1': d[6][0][2], 'pos_1_7_2': d[6][0][3],
            'pos_1_8_1': d[7][0][2], 'pos_1_8_2': d[7][0][3],
            'pos_1_9_1': d[8][0][2], 'pos_1_9_2': d[8][0][3],
            'pos_1_10_1': d[9][0][2], 'pos_1_10_2': d[9][0][3],
            'pos_1_11_1': d[10][0][2], 'pos_1_11_2': d[10][0][3],
            'pos_1_12_1': d[11][0][2], 'pos_1_12_2': d[11][0][3],
            'pos_1_13_1': d[12][0][2], 'pos_1_13_2': d[12][0][3],
            'pos_1_14_1': d[13][0][2], 'pos_1_14_2': d[13][0][3],
            'pos_1_15_1': d[14][0][2], 'pos_1_15_2': d[14][0][3],
            'pos_1_16_1': d[15][0][2], 'pos_1_16_2': d[15][0][3],
            'pos_1_17_1': d[16][0][2], 'pos_1_17_2': d[16][0][3],
            'pos_1_18_1': d[17][0][2], 'pos_1_18_2': d[17][0][3],
            'pos_1_19_1': d[18][0][2], 'pos_1_19_2': d[18][0][3],
            'pos_1_20_1': d[19][0][2], 'pos_1_20_2': d[19][0][3],
            'pos_1_21_1': d[20][0][2], 'pos_1_21_2': d[20][0][3],
            'pos_1_22_1': d[21][0][2], 'pos_1_22_2': d[21][0][3],
            'pos_1_23_1': d[22][0][2], 'pos_1_23_2': d[22][0][3],
            'pos_1_24_1': d[23][0][2], 'pos_1_24_2': d[23][0][3],
            'pos_1_25_1': d[24][0][2], 'pos_1_25_2': d[24][0][3],
            'pos_1_26_1': d[25][0][2], 'pos_1_26_2': d[25][0][3],
            'pos_1_27_1': d[26][0][2], 'pos_1_27_2': d[26][0][3],
            'pos_1_28_1': d[27][0][2], 'pos_1_28_2': d[27][0][3],
            'pos_1_29_1': d[28][0][2], 'pos_1_29_2': d[28][0][3],
            'pos_1_30_1': d[29][0][2], 'pos_1_30_2': d[29][0][3],
            #OF values of the second feature (tool_2)
            'feature_2_1_1': d[0][1][0], 'feature_2_1_2': d[0][1][1],
            'feature_2_2_1': d[1][1][0], 'feature_2_2_2': d[1][1][1],
            'feature_2_3_1': d[2][1][0], 'feature_2_3_2': d[2][1][1],
            'feature_2_4_1': d[3][1][0], 'feature_2_4_2': d[3][1][1],
            'feature_2_5_1': d[4][1][0], 'feature_2_5_2': d[4][1][1],
            'feature_2_6_1': d[5][1][0], 'feature_2_6_2': d[5][1][1],
            'feature_2_7_1': d[6][1][0], 'feature_2_7_2': d[6][1][1],
            'feature_2_8_1': d[7][1][0], 'feature_2_8_2': d[7][1][1],
            'feature_2_9_1': d[8][1][0], 'feature_2_9_2': d[8][1][1],
            'feature_2_10_1': d[9][1][0], 'feature_2_10_2': d[9][1][1],
            'feature_2_11_1': d[10][1][0], 'feature_2_11_2': d[10][1][1],
            'feature_2_12_1': d[11][1][0], 'feature_2_12_2': d[11][1][1],
            'feature_2_13_1': d[12][1][0], 'feature_2_13_2': d[12][1][1],
            'feature_2_14_1': d[13][1][0], 'feature_2_14_2': d[13][1][1],
            'feature_2_15_1': d[14][1][0], 'feature_2_15_2': d[14][1][1],
            'feature_2_16_1': d[15][1][0], 'feature_2_16_2': d[15][1][1],
            'feature_2_17_1': d[16][1][0], 'feature_2_17_2': d[16][1][1],
            'feature_2_18_1': d[17][1][0], 'feature_2_18_2': d[17][1][1],
            'feature_2_19_1': d[18][1][0], 'feature_2_19_2': d[18][1][1],
            'feature_2_20_1': d[19][1][0], 'feature_2_20_2': d[19][1][1],
            'feature_2_21_1': d[20][1][0], 'feature_2_21_2': d[20][1][1],
            'feature_2_22_1': d[21][1][0], 'feature_2_22_2': d[21][1][1],
            'feature_2_23_1': d[22][1][0], 'feature_2_23_2': d[22][1][1],
            'feature_2_24_1': d[23][1][0], 'feature_2_24_2': d[23][1][1],
            'feature_2_25_1': d[24][1][0], 'feature_2_25_2': d[24][1][1],
            'feature_2_26_1': d[25][1][0], 'feature_2_26_2': d[25][1][1],
            'feature_2_27_1': d[26][1][0], 'feature_2_27_2': d[26][1][1],
            'feature_2_28_1': d[27][1][0], 'feature_2_28_2': d[27][1][1],
            'feature_2_29_1': d[28][1][0], 'feature_2_29_2': d[28][1][1],
            'feature_2_30_1': d[29][1][0], 'feature_2_30_2': d[29][1][1],
             #coordinates of the second feature (tool_2)
            'pos_2_1_1': d[0][1][2], 'pos_2_1_2': d[0][1][3],
            'pos_2_2_1': d[1][1][2], 'pos_2_2_2': d[1][1][3],
            'pos_2_3_1': d[2][1][2], 'pos_2_3_2': d[2][1][3],
            'pos_2_4_1': d[3][1][2], 'pos_2_4_2': d[3][1][3],
            'pos_2_5_1': d[4][1][2], 'pos_2_5_2': d[4][1][3],
            'pos_2_6_1': d[5][1][2], 'pos_2_6_2': d[5][1][3],
            'pos_2_7_1': d[6][1][2], 'pos_2_7_2': d[6][1][3],
            'pos_2_8_1': d[7][1][2], 'pos_2_8_2': d[7][1][3],
            'pos_2_9_1': d[8][1][2], 'pos_2_9_2': d[8][1][3],
            'pos_2_10_1': d[9][1][2], 'pos_2_10_2': d[9][1][3],
            'pos_2_11_1': d[10][1][2], 'pos_2_11_2': d[10][1][3],
            'pos_2_12_1': d[11][1][2], 'pos_2_12_2': d[11][1][3],
            'pos_2_13_1': d[12][1][2], 'pos_2_13_2': d[12][1][3],
            'pos_2_14_1': d[13][1][2], 'pos_2_14_2': d[13][1][3],
            'pos_2_15_1': d[14][1][2], 'pos_2_15_2': d[14][1][3],
            'pos_2_16_1': d[15][1][2], 'pos_2_16_2': d[15][1][3],
            'pos_2_17_1': d[16][1][2], 'pos_2_17_2': d[16][1][3],
            'pos_2_18_1': d[17][1][2], 'pos_2_18_2': d[17][1][3],
            'pos_2_19_1': d[18][1][2], 'pos_2_19_2': d[18][1][3],
            'pos_2_20_1': d[19][1][2], 'pos_2_20_2': d[19][1][3],
            'pos_2_21_1': d[20][1][2], 'pos_2_21_2': d[20][1][3],
            'pos_2_22_1': d[21][1][2], 'pos_2_22_2': d[21][1][3],
            'pos_2_23_1': d[22][1][2], 'pos_2_23_2': d[22][1][3],
            'pos_2_24_1': d[23][1][2], 'pos_2_24_2': d[23][1][3],
            'pos_2_25_1': d[24][1][2], 'pos_2_25_2': d[24][1][3],
            'pos_2_26_1': d[25][1][2], 'pos_2_26_2': d[25][1][3],
            'pos_2_27_1': d[26][1][2], 'pos_2_27_2': d[26][1][3],
            'pos_2_28_1': d[27][1][2], 'pos_2_28_2': d[27][1][3],
            'pos_2_29_1': d[28][1][2], 'pos_2_29_2': d[28][1][3],
            'pos_2_30_1': d[29][1][2], 'pos_2_30_2': d[29][1][3],
            }
            #and add that to the csv as a row ->resulting in as many rows as frames
            writer.writerow(temp)

In [13]:
novice_list = { 'B', 'G', 'H', 'I'}
interm_list = { 'C', 'F'}
expert_list = { 'D', 'E'}
import csv

def saveRawDataAccordingToExpertise(novices, interms, experts, user, attempt,data, novice_ind, interm_ind, expert_ind):
    if data == []:
        return novices, interms, experts, novice_ind, interm_ind, expert_ind
    if user in novice_list:
        novices[novice_ind] = np.array([
            {
                'data': data,
                'userid': user,
                'attemptid':attempt
            }
        ])
        novice_ind += 1
    if user in interm_list:
        interms[interm_ind] = np.array([
            {
                'data': data,
                'userid': user,
                'attemptid':attempt
            }
        ])
        interm_ind += 1
    if user in expert_list:
        experts[expert_ind] = np.array([
            {
                'data': data,
                'userid': user,
                'attemptid':attempt
            }
        ])
        expert_ind += 1
    return novices, interms, experts, novice_ind, interm_ind, expert_ind


def getMinFrameRange(data): #not used, too much data loss
    val = int(data[2][0]) - int(data[1][0])
    for i in range(len(data[0])):
        diff = int(data[2][i]) - int(data[1][i])
        if diff < val:
            val = diff
    return val
            
def extractDataFromVideoList(vlist, skill):
    path = root + skill + '/video/'
    temp = {}
    novices = {}
    interms = {}
    experts = {}
    novice_ind, interm_ind, expert_ind = 0,0,0
    i = 0
    v_ind = 0
    for v in vlist:
        v_ind += 1
        user, attempt, capture, skill = parseFileName(v)
        input_path = path + v
        cap = cv.VideoCapture(input_path)
        if not cap.isOpened():
            print('Unable to open: ' + input_path)
            exit(0)

        p0, actual_frame_begin = getROIdata(skill, user, attempt, capture)

        _, frame1 = cap.read()
        prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
        cap.release()
        if separateGestures:
            gpath = TranscriptionPath(user, attempt, skill)
            gestureData = getGestureSeparatedFrameRanges(gpath)
            min_frame_range = getMinFrameRange(gestureData)
            for g_ind in range(len(gestureData[0])):
                gesture = gestureData[0][g_ind]
                if g_ind == 0:
                    frame_begin = actual_frame_begin
                else:
                    frame_begin = gestureData[1][g_ind]
                frame_end = gestureData[2][g_ind]
                if saveRawOF:
                    flow = extractDataFromSpecificFrames(path + v, v, gesture,frame_begin, frame_end, p0, min_frame_range)
                    if saveCSV:
                        saveCSVs(skill, user, attempt, capture, flow)
                    else:
                        #for f in flow:
                        #    novices, interms, experts, novice_ind, interm_ind, expert_ind = saveRawDataAccordingToExpertise(novices, interms, experts, user, attempt,f, novice_ind, interm_ind, expert_ind)
                        novices, interms, experts, novice_ind, interm_ind, expert_ind = saveRawDataAccordingToExpertise(novices, interms, experts, user, attempt,flow, novice_ind, interm_ind, expert_ind)
                else:
                    histo, p0 = extractDataFromSpecificFrames(path + v, v, gesture,frame_begin, frame_end, p0)
                    temp[i] = np.array([
                        {'user': str(user),
                         'attempt_id': attempt,
                         'data':histo,
                         'gesture':str(gesture),
                        }], dtype=np.object)
        else:
            cap = cv.VideoCapture(input_path)
            frame_end = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
            if saveRawOF:
                flow = extractDataFromSpecificFrames(path + v, v, "no_gesture", actual_frame_begin, frame_end, p0, 0)
                if saveCSV:
                    saveCSVs(skill, user, attempt, capture, flow)
                else:
                    #for f in flow:
                    #   novices, interms, experts, novice_ind, interm_ind, expert_ind = saveRawDataAccordingToExpertise(novices, interms, experts, user, attempt,f, novice_ind, interm_ind, expert_ind)
                    novices, interms, experts, novice_ind, interm_ind, expert_ind = saveRawDataAccordingToExpertise(novices, interms, experts, user, attempt,flow, novice_ind, interm_ind, expert_ind)
            else:
                histo, p0 = extractDataFromSpecificFrames(path + v, v, "no_gesture", actual_frame_begin, frame_end, p0, 0)
                temp[i] = np.array([
                    {'user': str(user),
                     'attempt_id': attempt,
                     'data':histo}], dtype=np.object)
        i += 1
        print(str(v_ind) +' out of '+ str(len(vlist))+' videos processed, grouped and stored...')

    if saveRawOF and not saveCSV:
        output = {}
        output[1] = np.array(list(novices.items()))
        output[2] = np.array(list(interms.items()))
        output[3] = np.array(list(experts.items()))
        return{'dataX': list(output.items())}
    elif not saveRawOF:
         return{'dataA': list(temp.items())}

In [14]:
import scipy.io as sio
def writeMatFile(dictionary, filename):
    if saveRawOF:
        path = output_root + filename + '_dataX_basic_' + str(180) + '_' + str(60) + '.mat'
        sio.savemat(path, dictionary)
    else:
        path = output_root + filename
        sio.savemat(path, dictionary)
    print(path + ' written')

In [15]:
def createNestedDictionariesPerGestures(knots, needles, sutures):
    if saveRawOF:
        knot_dic = extractDataFromVideoList(knots, 'Knot_Tying')
        if not saveCSV:
            writeMatFile(knot_dic,'Knot')
        needle_dic = extractDataFromVideoList(needles, 'Needle_Passing')
        if not saveCSV:
            writeMatFile(needle_dic, 'Needle')
        suture_dic = extractDataFromVideoList(sutures, 'Suturing')
        if not saveCSV:
            writeMatFile(suture_dic, 'Suture')  
    else:
        knot_dic = extractDataFromVideoList(knots, 'Knot_Tying')
        writeMatFile(knot_dic, outfile_base + 'Knot.mat')
        needle_dic = extractDataFromVideoList(needles, 'Needle_Passing')
        writeMatFile(needle_dic, outfile_base + 'Needle.mat')
        suture_dic = extractDataFromVideoList(sutures, 'Suturing')
        writeMatFile(suture_dic, outfile_base + 'Suture.mat')  

In [16]:
createNestedDictionariesPerGestures(knot_list, needle_list, suture_list)

1 out of 72 videos processed, grouped and stored...
2 out of 72 videos processed, grouped and stored...
3 out of 72 videos processed, grouped and stored...
4 out of 72 videos processed, grouped and stored...
5 out of 72 videos processed, grouped and stored...
6 out of 72 videos processed, grouped and stored...
7 out of 72 videos processed, grouped and stored...
8 out of 72 videos processed, grouped and stored...
9 out of 72 videos processed, grouped and stored...
10 out of 72 videos processed, grouped and stored...
11 out of 72 videos processed, grouped and stored...
12 out of 72 videos processed, grouped and stored...
13 out of 72 videos processed, grouped and stored...
14 out of 72 videos processed, grouped and stored...
15 out of 72 videos processed, grouped and stored...
16 out of 72 videos processed, grouped and stored...
17 out of 72 videos processed, grouped and stored...
18 out of 72 videos processed, grouped and stored...
19 out of 72 videos processed, grouped and stored...
20

29 out of 78 videos processed, grouped and stored...
30 out of 78 videos processed, grouped and stored...
31 out of 78 videos processed, grouped and stored...
32 out of 78 videos processed, grouped and stored...
33 out of 78 videos processed, grouped and stored...
34 out of 78 videos processed, grouped and stored...
35 out of 78 videos processed, grouped and stored...
36 out of 78 videos processed, grouped and stored...
37 out of 78 videos processed, grouped and stored...
38 out of 78 videos processed, grouped and stored...
39 out of 78 videos processed, grouped and stored...
40 out of 78 videos processed, grouped and stored...
41 out of 78 videos processed, grouped and stored...
42 out of 78 videos processed, grouped and stored...
43 out of 78 videos processed, grouped and stored...
44 out of 78 videos processed, grouped and stored...
45 out of 78 videos processed, grouped and stored...
46 out of 78 videos processed, grouped and stored...
47 out of 78 videos processed, grouped and sto